## Time ordered interactions
Produces an interaction list from a ```.mymridon``` experiment file and saves it as a csv. You can specify different matchers (interactions with specific antIDs, interactions of specific body parts) and output this into a csv with the time of each interaction.

In [ ]:
from datetime import datetime, timedelta

import pandas as pd
import py_fort_myrmidon as fm

In [ ]:
def output_focal_interactions(exp, antID_list, start_time, duration_hours):
    # Create timestamps
    t_start = datetime(
        start_time[0], start_time[1], start_time[2], start_time[3], start_time[4]
    ).astimezone(tz=None)
    t_end = t_start + timedelta(hours=duration_hours)
    # Create matcher
    focal_IDs = antID_list
    # Create Or matcher to select interactions between head (1) and leg (3)
    focal_interaction_types = fm.Matcher.Or(
        fm.Matcher.InteractionType(1, 3), fm.Matcher.InteractionType(3, 1)
    )
    # Create matchers for each antID, interaction type combination
    focal_interactions = [
        fm.Matcher.And(fm.Matcher.AntID(x), focal_interaction_types) for x in focal_IDs
    ]
    # Create single matcher object by unpacking the list within an Or Matcher
    focal_matches = fm.Matcher.Or(*focal_interactions)
    # Output trajectories
    trajectories, interactions = fm.Query.ComputeAntInteractions(
        exp, start=t_start, end=t_end, matcher=focal_matches
    )
    # Get a list of lists with details of each interaction
    l_interaction = []
    for ia in interactions:
        l_interaction.append(
            list(ia.IDs)
            + [ia.Start, ia.End, ia.Space]
            + [(ia.End - ia.Start).Seconds()]
            + [any((ia.Types[:] == [1, 3]).all(1)), any((ia.Types[:] == [3, 1]).all(1))]
        )
    # This part can be replaced with the alternate code by matthias which may be faster
    # # get antIDs from the interaction object
    # antIDs = [x.IDs for x in interactions]
    # # Unpack the list
    # antID1, antID2 = map(list, zip(*antIDs))
    # # Obtain start times of the interactions
    # start_times = [str(x.Start) for x in interactions]
    # # Obtain duration of interactions
    # duration = [(x.End - x.Start).Seconds() for x in interactions]
    interaction_df = pd.DataFrame(
        l_interaction,
        columns=[
            "antID1",
            "antID2",
            "start",
            "end",
            "space",
            "duration(s)",
            "1-3",
            "3-1",
        ],
    )
    interaction_df.to_csv(
        "focal_ant_interaction_times_{}_{}_{}.csv".format(exp.Name, t_start, t_end),
        index=False,
    )

## Colony 54

In [ ]:
f_myrmidon = "/media/egeorge/Redhood/Experiment3/woundcare_cfell54_T3.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)

In [ ]:
# output_focal_interactions([106,22], (2022, 5, 2, 16, 3), 6)
focal_ants = [[108, 107], [114, 9], [62, 87], [12, 83], [53, 101]]
start_times = [
    (2022, 6, 20, 14, 35),
    (2022, 6, 21, 14, 21),
    (2022, 6, 22, 14, 28),
    (2022, 6, 23, 14, 14),
    (2022, 6, 24, 14, 31),
]

In [ ]:
[output_focal_interactions(x, y, 6) for x, y in zip(exp, focal_ants, start_times)]